### txt 파일로 데이터 프레임 만들고 토큰화 등 전처리

In [1]:
import pandas as pd
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import text_to_word_sequence

In [2]:
train_data = pd.read_table('result.txt', header = None) # txt파일로 데이터 테이블 생성

In [3]:
train_data.columns = ['document'] # 컬럼 명을 정해주고

In [4]:
 train_data # 출력해보기

,document
0,Oxygen is the chemical element with the symbol...
1,Oxygen was isolated by Michael Sendivogius bef...
2,Common uses of oxygen include production of st...
3,History of study
4,Early experiments
...,...
96,"Oxygen (O2) Properties, Uses, Applications"
97,"Roald Hoffmann article on ""The Story of O"""
98,WebElements.com – Oxygen
99,Oxygen on In Our Time at the BBC


In [5]:
# NULL 값 존재 유무
print(train_data.isnull().values.any())

False


In [6]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [7]:
features = train_data['document'].tolist() # 데이터 프레임의 'documnet' 의 내용을
                                            # 리스트로 변환

In [8]:
parse_text = "".join([str(_) for _ in features]) #리스트로 변환 된 것을 문자열로 변환

In [9]:
content_text = re.sub(r'\([^)]*\)', '', parse_text) # 괄호로 구성된 내용 제거

In [10]:
sent_text = sent_tokenize(content_text) # lntk를 이용하여 문장 토큰화 수행.

In [11]:
# 각 문장에 대해 구두점 제거 밑 대문자 -> 소문자로 변환

normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

In [12]:
# 각 문장에 대해 nltk를 이용하여 단어 토큰화 수행

result = [word_tokenize(sentence) for sentence in normalized_text]

In [13]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 233


In [14]:
# 샘플 3개만 출력
for line in result[:3]:
    print(line)

['oxygen', 'is', 'the', 'chemical', 'element', 'with', 'the', 'symbol', 'o', 'and', 'atomic', 'number', '8']
['it', 'is', 'a', 'member', 'of', 'the', 'chalcogen', 'group', 'in', 'the', 'periodic', 'table', 'a', 'highly', 'reactive', 'nonmetal', 'and', 'an', 'oxidizing', 'agent', 'that', 'readily', 'forms', 'oxides', 'with', 'most', 'elements', 'as', 'well', 'as', 'with', 'other', 'compounds']
['oxygen', 'is', 'earth', 's', 'most', 'abundant', 'element', 'and', 'after', 'hydrogen', 'and', 'helium', 'it', 'is', 'the', 'third', 'most', 'abundant', 'element', 'in', 'the', 'universe']


### Word2Vec 훈련시키기
size- = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.<br>
window = 컨텍스트 윈도우 크기<br>
min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)<br>
workers = 학습을 위한 프로세스 수<br>
sg = 0은 CBOW, 1은 Skip-gram.<br>

In [15]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)

In [16]:
model_result = model.wv.most_similar("chemical")
print(model_result)

[('and', 0.9992905855178833), ('of', 0.9991757869720459), ('in', 0.9991697072982788), ('a', 0.9991617202758789), ('is', 0.9991567730903625), ('oxygen', 0.9991112351417542), ('to', 0.9990828037261963), ('the', 0.9990788698196411), ('2', 0.9990768432617188), ('by', 0.9990584850311279)]


In [17]:
model.wv.save_word2vec_format('eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드

### 데이터 더 모아서 학습시키면 더 좋은 결과 얻을 수 있음

In [45]:
model_result = loaded_model.most_similar("chemical")
print(model_result)

[('and', 0.9992905855178833), ('of', 0.9991757869720459), ('in', 0.9991697072982788), ('a', 0.9991617202758789), ('is', 0.9991567730903625), ('oxygen', 0.9991112351417542), ('to', 0.9990828037261963), ('the', 0.9990788698196411), ('2', 0.9990768432617188), ('by', 0.9990584850311279)]


### word2word로 영어 번역

In [47]:
from word2word import Word2word

In [48]:
en2kr = Word2word('en', 'ko')

kr2en = Word2word('ko', 'en')
print(en2kr(model_result[0][0]))

['and', '해서', '랑', '모든', '집']


### 근데 우리가 하려는게 이런 거 아니었나...;;

In [49]:
en2kr('oxygen') # ㅋㅋㅋㅋ

['산소', '공급', '이산화', '탄소', '호흡기']